In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [2]:
train = pd.read_csv('train.csv',parse_dates = ['Dates'])
test = pd.read_csv('test.csv',parse_dates = ['Dates'])

In [3]:
train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [39]:
from collections import Counter
category_encoder = LabelEncoder()
category = category_encoder.fit_transform(train.Category)
days = pd.get_dummies(train.DayOfWeek)
district = pd.get_dummies(train.PdDistrict)
hour = pd.get_dummies(train.Dates.dt.hour)
month = pd.get_dummies(train.Dates.dt.month)
x = train.X
y = train.Y
month=pd.get_dummies(pd.concat([train.Dates.dt.month, test.Dates.dt.month]))
month_tr = month[:train.shape[0]]
month_te = month[train.shape[0]:]
day=train.Dates.dt.dayofyear
num_add = Counter(train.Address)
address = train.Address.apply(lambda x: num_add[x])
streetno = train.Address.apply(lambda x: x.split(' ', 1)[0] if x.split(' ', 1)[0].isdigit() else 0)
road = train.Address.apply(lambda x: x.split(' ', 1)[1] if x.split(' ', 1)[0].isdigit() else x)
night = train.Dates.apply(lambda x: 1 if x.hour >= 18 or x.hour <= 4 else 0)
train_data = pd.concat([hour, days, day, district, address, month_tr], axis=1)
train_data['Target'] = category
train_labels=category

days = pd.get_dummies(test.DayOfWeek)
district = pd.get_dummies(test.PdDistrict)
hour = pd.get_dummies(test.Dates.dt.hour)
num_add = Counter(test.Address)
address = test.Address.apply(lambda x: num_add[x])
day=test.Dates.dt.dayofyear
# ￼x=test.X
# y=test.Y
night = test.Dates.apply(lambda x: 1 if x.hour >= 18 or x.hour <= 4 else 0)
test_data = pd.concat([hour, days, day, district, address, month_te], axis=1)

(878049, 9) (878049, 12)


In [19]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN,Address
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,65
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,198
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,20
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,450
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,450


In [40]:
# validation (optional)
training, validation = train_test_split(train_data, train_size=.60)
model = LogisticRegression(C=.01)
model.fit(training.ix[:,:-1], training.Target)
predicted = np.array(model.predict_proba(validation.ix[:,:-1]))
log_loss(validation.Target, predicted)

2.5886131691394096

In [ ]:
training, validation = train_test_split(train_data, train_size=.60)
model = RandomForestClassifier(n_estimators=200, max_depth=20, n_jobs=-1)
model.fit(training.ix[:,:-1], training.Target)
predicted = np.array(model.predict_proba(validation.ix[:,:-1]))
log_loss(validation.Target, predicted)

In [21]:
xg_train = xgb.DMatrix( train_data.values, label=category)
xg_test = xgb.DMatrix(test_data.values)
param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.1
param['max_depth'] = 15
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = len(set(category))
watchlist = [ (xg_train,'train')]
num_round = 10
bst = xgb.train(param, xg_train, num_round, watchlist )
yprob = bst.predict( xg_test )
print(yprob.shape)
# log_loss(validation.Target, yprob)

[0]	train-merror:0.703641
[1]	train-merror:0.692217
[2]	train-merror:0.685027
[3]	train-merror:0.679313
[4]	train-merror:0.674180
[5]	train-merror:0.669831
[6]	train-merror:0.665431
[7]	train-merror:0.661718
[8]	train-merror:0.658291
[9]	train-merror:0.655182


(884262, 39)


/usr/local/lib/python3.4/dist-packages/xgboost/core.py:840: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  preds = preds.reshape(nrow, preds.size / nrow)


In [64]:
model = LogisticRegression(C=.01)
model.fit(train_data, category)
predicted = np.array(model.predict_proba(test_data))

In [22]:
result=pd.DataFrame(yprob, columns=category_encoder.classes_)
result.to_csv('sub.csv', index = True, index_label = 'Id' )

In [5]:
# some manual feature engineering
train['StreetNo'] = train['Address'].apply(lambda x: x.split(' ', 1)[0] if x.split(' ', 1)[0].isdigit() else 0)
train['Address'] = train['Address'].apply(lambda x: x.split(' ', 1)[1] if x.split(' ', 1)[0].isdigit() else x)
test['StreetNo'] = test['Address'].apply(lambda x: x.split(' ', 1)[0] if x.split(' ', 1)[0].isdigit() else 0)
test['Address'] = test['Address'].apply(lambda x: x.split(' ', 1)[1] if x.split(' ', 1)[0].isdigit() else x)
train['dark'] = train['Dates'].apply(lambda x: 1 if int(x.split(' ')[1].split(':')[0]) >= 18 or 
                                     int(x.split(' ')[1].split(':')[0]) <= 4 else 0)
test['dark'] = test['Dates'].apply(lambda x: 1 if int(x.split(' ')[1].split(':')[0]) >= 18 or
                                     int(x.split(' ')[1].split(':')[0]) <= 4 else 0)
# train.drop('Dates', axis=1, inplace=True)
# test.drop('Dates', axis=1, inplace=True)
train.drop('Descript', axis=1, inplace=True)
train.drop('Resolution', axis=1, inplace=True)
labels = train['Category']
idd = test['Id']
train.drop('Category', axis=1, inplace=True)
test.drop('Id', axis=1, inplace=True)

In [6]:
# non-numeric to numeric
numeric = ['X', 'Y', 'StreetNo', 'dark']
lb = LabelEncoder()
for column in train.columns:
    if column not in numeric: 
        if column != 'Category':
            lb.fit(list(train[column])+list(test[column]))
            test[column] = lb.transform(test[column])
            train[column] = lb.transform(train[column])

In [7]:
# scaling
scaler = StandardScaler()
for column in test.columns:
    scaler.fit(list(train[column])+list(test[column]))
    train[column] = scaler.transform(train[column])
    test[column] = scaler.transform(test[column])
# scaler.fit(list(train.ix[:,1:].values)+list(test.ix[:,1:].values))
# train.ix[:,1:] = scaler.transform(train.ix[:,1:])
# test.ix[:,1:] = scaler.transform(test.ix[:,1:])

/usr/local/lib/python3.4/dist-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/va

In [43]:
category_encoder=LabelBinarizer()
onehotlabels = category_encoder.fit_transform(category)

In [44]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(30, input_dim=train_data.shape[1], init='uniform'))
model.add(Activation('relu'))
model.add(Dense(70, init='uniform'))
model.add(Activation('relu'))
model.add(Dense(50, init='uniform'))
model.add(Activation('relu'))
model.add(Dense(39, init='uniform'))
model.add(Activation('softmax'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer='sgd')

model.fit(train_data.values, onehotlabels,
          nb_epoch=10, batch_size=1000)
# score = model.evaluate(X_test, y_test, batch_size=16)

Epoch 1/10
878049/878049 [==============================] - 12s - loss: 2.7638    
Epoch 2/10
878049/878049 [==============================] - 11s - loss: 2.6818    
Epoch 3/10
878049/878049 [==============================] - 11s - loss: 2.6815    
Epoch 4/10
209000/878049 [======>.......................] - ETA: 9s - loss: 2.6832

KeyboardInterrupt: 

In [11]:
p = model.predict_classes(test.values)
p = category_encoder.inverse_transform(p)

884262/884262 [==============================] - 62s    


ValueError: axis(=1) out of bounds

In [12]:
set(p)

{1, 4, 7, 16, 19, 20, 21, 23, 35, 36}

In [17]:
model2 = SGDClassifier(alpha=0.001, n_jobs=-1, n_iter=20,loss='log')
model2.fit(train.values, labels)

SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=20, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [20]:
p=model2.predict(test.values)
set(p)

{'ASSAULT',
 'BURGLARY',
 'DRUG/NARCOTIC',
 'LARCENY/THEFT',
 'OTHER OFFENSES',
 'VEHICLE THEFT'}

In [12]:
predicted_category = category_encoder.inverse_transform(p)

In [34]:
t = 'ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,EXTORTION,FAMILY OFFENSES,FORGERY/COUNTERFEITING,FRAUD,GAMBLING,KIDNAPPING,LARCENY/THEFT,LIQUOR LAWS,LOITERING,MISSING PERSON,NON-CRIMINAL,OTHER OFFENSES,PORNOGRAPHY/OBSCENE MAT,PROSTITUTION,RECOVERED VEHICLE,ROBBERY,RUNAWAY,SECONDARY CODES,SEX OFFENSES FORCIBLE,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS'
lb = LabelBinarizer().fit(t.split(','))

In [35]:
import csv
f = csv.writer(open('sub.csv','w'))
f.writerow(['Id']+t.split(','))
for i in range(test.shape[0]):
    t = list(lb.transform([p[i]])[0])
    f.writerow(idd[i]+t)